<a href="https://colab.research.google.com/github/MarySelifanova/Aminoacid-translocations/blob/main/130321_add_dictionaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add dictionaries for all states

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Install and import modules**

In [ ]:
!pip install bio
!pip install XlsxWriter
from Bio import SeqIO
from itertools import combinations
import numpy as np
import re
import pandas as pd
import xlsxwriter

**Load dataframes from tables**

In [16]:
#open the table you want to change
xls = pd.ExcelFile('/content/drive/MyDrive/Translocations/H1N1_H3N2.1557.AA.aligned.1str_pdb_multiple.xlsx')
df1 = pd.read_excel(xls, 'len=1')
df2 = pd.read_excel(xls, 'len=2')
df3 = pd.read_excel(xls, 'len=3')
df4 = pd.read_excel(xls, 'len=4')

In [17]:
df_list = [df1, df2, df3, df4]

In [18]:
def add_dictionaries(file, k, df, state): #file – alignment, k – length of kmer, df – table to change, state – '00'/'11'/'01'/'10'

  d_state_list = [] #create an empty list of state dictionaries to add into our dataframe

  for index1, row1 in df.iterrows(): #iterate through correlating pairs in dataframe
      d_state = {} #create an empty dictionary for the pair we're lookinng at 

      if int(row1[state]) != 0:

        p1 = int(row1['pos1'])
        p2 = int(row1['pos2'])
        kmer = row1['kmer']

        with open(file, "rU") as handle:  
          for record in SeqIO.parse(handle, "fasta"): #iterating through sequences in the alignment file
            test_str = str(record.seq) #test_str – one sequence; res – list of kmers of particular length k

            pos1_kmer = test_str[p1-1:p1+k-1] #find "translocation" kmer-1 of this pair i a particular sequence of the alignment
            pos2_kmer = test_str[p2-1:p2+k-1] #find "translocation" kmer-2 of this pair i a particular sequence of the alignment
            pair = pos1_kmer + '_' + pos2_kmer #form a pair 

            #undate state dictionary for currennt tarnnslocation pair with current sequece state

            if state == '00': 
              if pos1_kmer != kmer and pos2_kmer != kmer: #check '00' condition 

                if pair in d_state.keys():
                  d_state[pair] += 1
                else:
                  d_state[pair] = 1  

            elif state == '11':
              if pos1_kmer == kmer and pos2_kmer == kmer: #check '11' condition

                if pair in d_state.keys():
                  d_state[pair] +=1
                else:
                  d_state[pair] = 1

            elif state == '01':
              if pos1_kmer != kmer and pos2_kmer == kmer: #check '01' condition

                if pair in d_state.keys():
                  d_state[pair] +=1
                else:
                  d_state[pair] = 1

            elif state == '10':
              if pos1_kmer == kmer and pos2_kmer != kmer: #check '10' condition

                if pair in d_state.keys():
                  d_state[pair] +=1
                else:
                  d_state[pair] = 1
        

      d_state_list.append(d_state) #add pair dictionary to the list of state dictionaries 

  col_name = state + '_dictionary' #create column name
  df[col_name] = d_state_list #add column with dictionaries on particular state
  return df

In [19]:
#run add_dictionaries finction for all tables and states

file = '/content/drive/MyDrive/Translocations/H1N1_H3N2.1557.AA.aligned.1str.fasta'
df_list_updated = [] #create an empty list for new dataframes

for i in range(len(df_list)):
  d = df_list[i]
  k = i + 1

  d_new1 = add_dictionaries(file, k, d, '00')
  d_new2 = add_dictionaries(file, k, d_new1, '01')
  d_new3 = add_dictionaries(file, k, d_new2, '10')
  d_new4 = add_dictionaries(file, k, d_new3, '11')

  df_list_updated.append(d_new4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: 'U' mode is deprecated
  


In [ ]:
df_list_updated[3]

## **Save tables**

In [22]:
outfile = '/content/drive/MyDrive/Translocations/H1N1_H3N2_pdb_multiple_dictionaries.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')

# Write each dataframe to a different worksheet
df_list_updated[0].to_excel(writer, sheet_name='len=1')
df_list_updated[1].to_excel(writer, sheet_name='len=2')
df_list_updated[2].to_excel(writer, sheet_name='len=3')
df_list_updated[3].to_excel(writer, sheet_name='len=4')

# Close the Pandas Excel writer and output the Excel file
writer.save()